In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
import pickle
import os
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Flatten, Dense
from tensorflow.keras import Model

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    print(gpu)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


### Training Data

In [3]:
all_data = []
all_labels = []

In [4]:

def file_open(file_path):
    try:
        with open(file_path, 'rb') as f:
            data_dic = pickle.load(f, encoding="latin1")
            all_data.append(data_dic["data"])
            all_labels.append(data_dic["labels"])
    except FileNotFoundError:
        print(f"File {file_path} not found")
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")

In [5]:
# total 10 batches
base_path = r"Imagenet32_train"
for i in range (2):
    file_path = os.path.join(base_path, f"train_data_batch_{i+1}")
    file_open(file_path)

In [6]:
flattened_labels = [item for sublist in all_labels for item in sublist]

In [7]:
all_data = np.vstack(all_data)
all_labels = np.array(flattened_labels) 

print(f"Data shape: {all_data.shape}")
print(f"Labels shape: {all_labels.shape}") 

Data shape: (256232, 3072)
Labels shape: (256232,)


In [8]:
data = pd.DataFrame({
    "image_data": list(all_data),
    "label": all_labels
})

In [9]:
chunk_size = 10000
for start in range(0, len(data), chunk_size):
    end = start + chunk_size
    data.loc[start:end, 'image_data'] = data.loc[start:end, 'image_data'] / 255
    data.loc[start:end,'image_data'] = data.loc[start:end,'image_data'].map(lambda x: (np.round(x, 2)))
    data.loc[start:end, 'image_data'] =  data.loc[start:end,'image_data'].apply(
    lambda x: np.array(x).reshape((3, 32, 32)).transpose(1, 2, 0)
)

In [10]:
data['label'].value_counts()

label
178    301
159    300
187    299
319    298
469    298
      ... 
42     167
99     156
207    156
529    151
31     137
Name: count, Length: 1000, dtype: int64

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Perform stratified split
train, valid = train_test_split(data, test_size=0.2, stratify=data['label'], random_state=42)

In [12]:
images_valid = np.array(valid['image_data'].tolist())  
labels_valid = np.array(valid['label'])  

In [13]:
file_path = r'Imagenet32_val\val_data'

with open(file_path, 'rb') as f:
    test_dict = pickle.load(f, encoding='latin1')

print(test_dict.keys())

dict_keys(['labels', 'data'])


In [14]:
test = pd.DataFrame({
    'image_data': list(test_dict['data']), 
    'label': test_dict['labels']
})

In [15]:
test['image_data'] = test['image_data'].apply(
    lambda x: np.array(x).reshape((3, 32, 32)).transpose(1, 2, 0)
)

In [16]:
test['image_data'] = test['image_data'] / 255.0

In [17]:
test['image_data'] = test['image_data'].map(lambda x: (np.round(x, 2)))

In [18]:
test_images = np.array(test['image_data'].tolist()) 
test_labels = np.array(test['label'])  

In [19]:
df = pd.DataFrame(train)
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

train_splits = []

for train_index, _ in skf.split(df, df['label']):
    train_split = df.iloc[train_index]
    train_splits.append(train_split)

for i, split in enumerate(train_splits):
    print(f"Training Split {i + 1}: {split.shape}")


Training Split 1: (163988, 2)
Training Split 2: (163988, 2)
Training Split 3: (163988, 2)
Training Split 4: (163988, 2)
Training Split 5: (163988, 2)


In [20]:
images_1 = np.array(train_splits[0]['image_data'].tolist())
labels_1 = np.array(train_splits[0]['label'])  

# images_2 = np.array(train_splits[1]['image_data'].tolist())
# labels_2 = np.array(train_splits[1]['label']) 

# images_3 = np.array(train_splits[2]['image_data'].tolist())
# labels_3 = np.array(train_splits[2]['label']) 

# images_4 = np.array(train_splits[3]['image_data'].tolist())
# labels_4 = np.array(train_splits[3]['label']) 

# images_5 = np.array(train_splits[4]['image_data'].tolist())
# labels_5 = np.array(train_splits[4]['label']) 

In [21]:
train_splits[0]['label'].value_counts()

label
178    193
159    192
319    191
187    191
726    190
      ... 
42     107
207    100
99     100
529     97
31      88
Name: count, Length: 1000, dtype: int64

In [22]:
def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x


input = Input(shape = (32,32,3))
x = Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = 'same')(input)
x = BatchNormalization()(x)
x = ReLU()(x)

x = mobilnet_block(x, filters = 64, strides = 1)
x = mobilnet_block(x, filters = 128, strides = 1)
x = mobilnet_block(x, filters = 128, strides = 2)
x = mobilnet_block(x, filters = 256, strides = 2)
x = mobilnet_block(x, filters = 512, strides = 1)
x = mobilnet_block(x, filters = 512, strides = 2)
for _ in range (5):
    x = mobilnet_block(x, filters = 512, strides = 1)
x = mobilnet_block(x, filters = 1024, strides = 2)
x = mobilnet_block(x, filters = 1024, strides = 1)
# x = AvgPool2D (pool_size = 8, strides = 1, data_format='channels_first')(x)
x = Flatten()(x)
output = Dense (units = 1000, activation = 'softmax')(x)
model = Model(inputs=input, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 32, 32, 32)        0         
                                                                 
 depthwise_conv2d (Depthwise  (None, 32, 32, 32)       320       
 Conv2D)                                                         
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128   

In [23]:
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [24]:
model_checkpoint = ModelCheckpoint('MobileNet.h5', save_best_only=True, monitor="val_loss")

In [25]:
images_1.shape

(163988, 32, 32, 3)

In [26]:
labels_1.shape

(163988,)

In [27]:
training = model.fit(images_1, labels_1, batch_size = 32,epochs= 10, validation_data=(images_valid, labels_valid), callbacks=[model_checkpoint])

Epoch 1/10
5125/5125 [==============================] - 277s 52ms/step - loss: nan - sparse_categorical_accuracy: 6.0980e-06 - val_loss: nan - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/10
5125/5125 [==============================] - 229s 45ms/step - loss: nan - sparse_categorical_accuracy: 0.0000e+00 - val_loss: nan - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 3/10
2843/5125 [===============>..............] - ETA: 2:20 - loss: nan - sparse_categorical_accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
# training = model.fit(images_2, labels_2, batch_size = 32,epochs= 10, validation_data=(images_valid, labels_valid), callbacks=[model_checkpoint])

In [ ]:
# training = model.fit(images_3, labels_3, batch_size = 32,epochs= 10, validation_data=(images_valid, labels_valid), callbacks=[model_checkpoint])

In [ ]:
# training = model.fit(images_4, labels_4, batch_size = 32,epochs= 10, validation_data=(images_valid, labels_valid), callbacks=[model_checkpoint])

In [ ]:
# training = model.fit(images_5, labels_5, batch_size = 32,epochs= 10, validation_data=(images_valid, labels_valid), callbacks=[model_checkpoint])

In [ ]:
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [ ]:
training = model.fit(images_1, labels_1, batch_size = 32,epochs= 40, validation_data=(images_valid, labels_valid), callbacks=[model_checkpoint])

In [35]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(images_1)

In [ ]:
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
training2 = model.fit(datagen.flow(images_1, labels_1, batch_size=32),epochs= 20, validation_data=(images_valid, labels_valid), callbacks=[model_checkpoint])

In [47]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(images_1)

In [ ]:
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
training3 = model.fit(datagen.flow(images_1, labels_1, batch_size=32),epochs= 5, validation_data=(images_valid, labels_valid), callbacks=[model_checkpoint])

In [ ]:
fig = plt.figure()
plt.plot(training2.history['sparse_categorical_accuracy'], color='blue', label='accuracy')
plt.plot(training2.history['val_sparse_categorical_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(training2.history['loss'], color='blue', label='loss')
plt.plot(training2.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

### Test Data

In [ ]:
model.evaluate(test_images, test_labels)

In [59]:
test_pred = model.predict(test_images)

In [60]:
test_pred = test_pred.argmax(axis=-1)

In [ ]:
f1_class = f1_score(test_labels, test_pred,average=None)
print(f1_class)

In [62]:
import os

In [63]:
model.save(os.path.join('models','ResNet.h5'))